In [ ]:
# Install necessary libraries
!pip install transformers PyPDF2 torch ipywidgets

# Import libraries
from transformers import AutoModelForCausalLM, AutoProcessor
from PyPDF2 import PdfReader
import torch
import timeit
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files

# Step 1: Load the Microsoft Phi-3-vision-128k-instruct Model and Processor
model_id = "microsoft/Phi-3-vision-128k-instruct"

device = "cuda" if torch.cuda.is_available() else "cpu"  # Check if GPU is available

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    cache_dir="/content/my_models/phi_3_vision",
    device_map="auto",  # Automatically selects the correct device (GPU/CPU)
    trust_remote_code=True,
    torch_dtype="auto",
    _attn_implementation="eager"
).to(device)

processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

# Function to extract text from PDF resume
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# Function to analyze resume with Phi-3-vision-128k-instruct
def analyze_resume_with_phi(resume_text, job_description):
    pre_prompt = (
        "You are an expert AI resume analyst with extensive knowledge of the job market, hiring practices, and skills required for various roles. "
        "Your goal is to provide a thorough analysis of the given resume and job description to help the applicant improve their chances of getting hired."
        "Please extract the following key information from the resume:"
        "- Personal details (name, contact info, etc.)"
        "- Educational background "
        "- Relevant work experience"
        "- Key skills and proficiencies"
        "- Notable projects, publications, or achievements"
        "Then, compare the applicant's background to the requirements listed in the job description. Identify any gaps or areas where the applicant's skills and experience do not fully align with the job requirements."
        "Provide specific, actionable recommendations for how the applicant can strengthen their resume and application to be a more competitive candidate. Suggest additional skills to learn, projects to complete, or ways to better highlight their existing qualifications."
        "Finally, summarize your analysis with a paragraph that captures the applicant's overall strengths and weaknesses relative to the job, and their likelihood of getting an interview if they implement your recommendations. Be honest but constructive in your feedback."
        "Please format your response with clear section headings for each area of analysis. Use bullet points for recommendations. Aim for a response of at least 300 words that provides a thorough, thoughtful assessment. Let me know if you need any clarification on the job description or have additional questions."
    )

    prompt_input = f"Resume: {resume_text} Job Description: {job_description}"

    # Create prompt using the processor
    messages = [
        {"role": "user", "content": f"{pre_prompt} {prompt_input} Assistant: "}
    ]
    prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    inputs = processor(prompt, return_tensors="pt").to(device)

    generation_args = {
        "max_new_tokens": 1000,
        "temperature": 0.0,
        "do_sample": False,
    }

    # Generate the response
    generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args)

    # Remove input tokens from the generated output
    generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
    response = processor.batch_decode(generate_ids, skip_special_tokens=True)[0]

    return response

# GUI elements using ipywidgets
resume_upload = widgets.FileUpload(
    accept='.pdf',
    multiple=False,
    description='Upload Resume',
    button_style='info'
)
job_description_text = widgets.Textarea(
    placeholder='Enter job description here...',
    description='Job Description:',
    layout=widgets.Layout(width='100%', height='150px'),
    style={'description_width': 'initial'}
)
submit_button = widgets.Button(
    description="Analyze Resume",
    button_style='success'
)
output_area = widgets.Output()

# Function to handle the user inputs
def on_submit_button_clicked(b):
    clear_output(wait=True)  # Clear previous outputs
    display(widgets.VBox([resume_upload, job_description_text, submit_button, output_area]))  # Redisplay widgets

    if resume_upload.value:
        # Save the uploaded PDF file
        resume_file_name = list(resume_upload.value.keys())[0]
        resume_file = resume_upload.value[resume_file_name]['content']
        with open(resume_file_name, 'wb') as f:
            f.write(resume_file)

        # Extract text from resume
        resume_text = extract_text_from_pdf(resume_file_name)

        # Get the job description
        job_description = job_description_text.value

        if not job_description:
            with output_area:
                print("Please enter a job description.")
            return

        # Start analysis
        start = timeit.default_timer()
        recommendations = analyze_resume_with_phi(resume_text, job_description)
        stop = timeit.default_timer()

        duration = stop - start

        # Display output
        with output_area:
            print("Time taken for analysis: {:.2f} seconds".format(duration))
            print("\n--- Recommendations ---\n")
            print(recommendations)

            # Save the result to a file
            with open("response.txt", "a") as f:
                f.write(f"Time taken: {duration:.2f} seconds\n")
                f.write(recommendations + "\n\n")

    else:
        with output_area:
            print("Please upload a resume PDF file.")

submit_button.on_click(on_submit_button_clicked)

# Display the widgets
display(widgets.VBox([resume_upload, job_description_text, submit_button, output_area]))


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
